In [1]:
#from our_dataset import ProofDataset
from our_dataset_with_labels import ProofDatasetWithLabels
from statement_embedding import emb_to_stmt, num_to_label
from torch_geometric.transforms import NormalizeFeatures
import numpy as np
import pandas as pd
import csv
import torch
from gensim.models import Word2Vec


In [195]:
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [3]:
# WARNING!!!!
# running ProofDataset(root="data/", file_limit=None) currently has a high initial cost
# it requires ~63 GB of space, and it took my computer about 2 hours to run
# however, after an initial run, the dataset doesn't need to be created again
# secondary runs merely verify that all files are in place; this takes my computer about 15s
# WARNING!!!!

# to avoid previous warning, use file_limit=8000 to only load and verify the first 8000 graphs (<4GB)
# if entire graph dataset is desired, use file_limit=None
file_limit = 20000    # desired number of graphs to work with
vocab_size = 1598   # number of characters in our vocabulary
label_size = file_limit  # number of labels in dataset; all labels is 45332; defaults to file_limit in pf_data

pf_data = ProofDatasetWithLabels(root="data/",file_limit=file_limit, label_size=file_limit )  # normalize with transform = NormalizeFeatures()

C:\Users\dphil\anaconda3\envs\erdos_may_2024\lib\site-packages\torch_geometric\data\dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.

In [7]:
# this function is used to get the label of a statement from tag_proof.csv

def get_thm_lbl(num):
    with open("../Assets/tag_proof.csv","r") as file:
        reader = csv.DictReader(file)
        for idx, row in enumerate(reader):
            if num == idx:
                return row['tag']

In [8]:
# let's explore graph 2, i.e. mp2, as our current graph at proof step 3
graph_num = 2
step_num = 3
graph_lbl = get_thm_lbl(graph_num)
cur_graph = pf_data.get(graph_num)
cur_step = cur_graph.x[step_num]

# let's get the normalized statement feature vector at step 3 in our proof
cur_step_vec_norm = cur_step
cur_step_vec_norm = cur_step_vec_norm[cur_step_vec_norm.nonzero()]  # remove trailing zeros
cur_step_vec_norm = cur_step_vec_norm.reshape(-1)
cur_step_vec_norm = cur_step_vec_norm.tolist()

# unnormalized version of above
cur_step_vec = cur_step*vocab_size
cur_step_vec = cur_step_vec[cur_step_vec.nonzero()] # remove trailing zeros
cur_step_vec = cur_step_vec.reshape(-1)
cur_step_vec = cur_step_vec.to(int).tolist()

# convert feature vector back to statement
#stmt_list = cur_step_vec.copy()
#stmt_list.insert(0,0)

# let's get the label of our current step
cur_label = cur_graph.y[step_num]*label_size
cur_label = cur_label.to(int).item()

# get statement dependencies of current step 
req_stmt_num = []
req_stmt = []
for idx, x in enumerate(cur_graph.edge_index[1]):
    if step_num == x:
        req_stmt_num.append(cur_graph.edge_index[0][idx].to(int))
for num in req_stmt_num:
    dependency = cur_graph.x[num]*vocab_size
    dependency = dependency.to(int).tolist()
    dependency.insert(0,0)
    dependency = emb_to_stmt(dependency)
    req_stmt.append(dependency)



C:\Users\dphil\OneDrive\Documents\proof-truth\proof-truth\Model\our_dataset_with_labels.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(osp.join(self.

In [296]:
#Gets the hypothesis for a given step
def hypos(cur_graph,step_num):
    req_stmt_num = []
    for idx, x in enumerate(cur_graph.edge_index[1]):
        if step_num == x:
            req_stmt_num.append(int(cur_graph.edge_index[0][idx]))
    return req_stmt_num

In [298]:
#Constructs a random walk up the graph of length less than 15
def graphwalks(cur_graph):
    walk=[cur_graph.num_nodes-1]
    start=cur_graph.num_nodes-1
    for i in range(15):
        hyp=hypos(cur_graph,start)
        if len(hyp)!=0:
            start=np.random.choice(hyp)
            walk.append(start)
    return walk

In [11]:
graphwalks(cur_graph)

[4, 3, 1]

In [300]:
#Creates a paragraph based on a random graph walk
def sentence(cur_graph):
    walk=graphwalks(cur_graph)
    sent=[]
    for step_num in reversed(walk):
        cur_step = cur_graph.x[step_num]
        cur_step_vec = cur_step*vocab_size
        cur_step_vec = cur_step_vec[cur_step_vec.nonzero()] # remove trailing zeros
        cur_step_vec = cur_step_vec.reshape(-1)
        sent.extend(cur_step_vec.to(int).tolist())
        #The special character -1, acts as a sentence break, telling the model that a statement has ended
        sent.append(-1)
    return sent         

In [87]:
sentences=[]

In [301]:
#Creates 4 sentences for each graph
for i in range(1,2139):
    for j in range(4):
        graph=pf_data.get(graph_num)
        sentences.append(sentence(graph))
    

C:\Users\dphil\OneDrive\Documents\proof-truth\proof-truth\Model\our_dataset_with_labels.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(osp.join(self.

KeyboardInterrupt: 

In [175]:
#constructs the dictionary of words.
dict={857}
for sent in sentences:
    for word in sent:
        dict.add(word)

In [179]:
dictionary=list(dict)

In [180]:
len(dictionary)

7

In [181]:
#Turns a word into a vector
def tovec(word):
    vec=np.zeros(7)
    for i in range(7):
        if word==dictionary[i]:
            vec[i]=1
    return vec
            

In [243]:
#converts our sentences to vectors
vecsents=[]
for sent in sentences:
    for i in range(3):
        vecsent=[]
        start=np.random.randint(1,len(sent))
        truncsent=sent[0:start]
        if len(truncsent)<10:
            for i in range(10-len(truncsent)):
                vecsent.append(np.zeros(len(dictionary)))
        for word in truncsent:
            vecsent.append(tovec(word))
            if len(vecsent)==10:
                break
    vecsents.append(np.array(vecsent))

In [244]:
vecsents[5].shape

(10, 7)

In [245]:
len(vecsents)

8552

In [246]:
datatens = torch.tensor(vecsents, dtype=torch.float32).reshape(8552, 10, 7)

In [247]:
#Constructs the skip-grams
dataX=datatens[:,0:9,:]
print(dataX.shape)
datay=datatens[:,9,:]
print(datay.shape)

torch.Size([8552, 9, 7])
torch.Size([8552, 7])


In [281]:
#defines the model
class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=7, hidden_size=100, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(100, 7)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x

In [282]:
#Trains the model
n_epochs = 20
batch_size = 120
model = CharModel()
 
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(dataX, datay), shuffle=True, batch_size=batch_size)
 
best_model = None
best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    loss = 0
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss += loss_fn(y_pred, y_batch)
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))
 
#torch.save([best_model, char_to_int], "single-char.pth")

Epoch 0: Cross-entropy: 12243.2637
Epoch 1: Cross-entropy: 5845.8550
Epoch 2: Cross-entropy: 3552.3167
Epoch 3: Cross-entropy: 2684.5513
Epoch 4: Cross-entropy: 2106.2007
Epoch 5: Cross-entropy: 1620.4266
Epoch 6: Cross-entropy: 1457.0802
Epoch 7: Cross-entropy: 1376.5580
Epoch 8: Cross-entropy: 1339.0958
Epoch 9: Cross-entropy: 1259.7552
Epoch 10: Cross-entropy: 1217.3639
Epoch 11: Cross-entropy: 1193.7926
Epoch 12: Cross-entropy: 1197.3599
Epoch 13: Cross-entropy: 1213.9332
Epoch 14: Cross-entropy: 1165.1077
Epoch 15: Cross-entropy: 1150.5404
Epoch 16: Cross-entropy: 1149.1710
Epoch 17: Cross-entropy: 1147.0646
Epoch 18: Cross-entropy: 1143.2030
Epoch 19: Cross-entropy: 1137.3346


In [283]:
#An example, which will probably work differently because of the randomization.
example=np.array([tovec(word) for word in sentences[24]])

In [284]:
example

array([[0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [291]:
extensor=torch.tensor(example[3:12], dtype=torch.float32).reshape(1, 9, 7)

In [292]:
extensor

tensor([[[0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0.],
         [0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0.]]])

In [293]:
dictionary

[705, 289, 1284, 114, 857, 1181, -1]

In [294]:
y_pred = model(extensor)

In [295]:
#The prediction, with my seed gives a high chance to 705 and -1. 705 (the slightly lower chance) turns out to be correct.
y_pred

tensor([[ 5.1864, -4.3440, -2.6406, -1.1165, -3.6526, -3.1844,  6.5438]],
       grad_fn=<AddmmBackward0>)